In [1]:
from data_import import *
from data_preprocess import *

In [2]:
NLP_TOPIC_RECOGNITION_FILEPATH = '../NLP Topic Recognition'
NLP_MODEL_FILEPATH = '../NLP Model Training/model_100K_glove'
TEXTUAL_RAW_DATA_FILEPATH = '../Data/textual_raw'

OUTPUT_DATASET_FILEPATH = "Dataset_sentiment_100K_glove_filtered_2g.csv"

In [3]:
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.append(NLP_TOPIC_RECOGNITION_FILEPATH)
from topic_recognition import *
#from NLP_class import *

*** Introductory Examples for the NLTK Book ***
Loading text1, ..., text9 and sent1, ..., sent9
Type the name of the text or sentence to view it.
Type: 'texts()' or 'sents()' to list the materials.
text1: Moby Dick by Herman Melville 1851
text2: Sense and Sensibility by Jane Austen 1811
text3: The Book of Genesis
text4: Inaugural Address Corpus
text5: Chat Corpus
text6: Monty Python and the Holy Grail
text7: Wall Street Journal
text8: Personals Corpus
text9: The Man Who Was Thursday by G . K . Chesterton 1908


In [4]:
from tensorflow import keras
model = keras.models.load_model(NLP_MODEL_FILEPATH)

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 50, 100)           10697600  
_________________________________________________________________
lstm (LSTM)                  (None, 100)               80400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 10,778,101
Trainable params: 80,501
Non-trainable params: 10,697,600
_________________________________________________________________


In [6]:
txt_folder = Path(TEXTUAL_RAW_DATA_FILEPATH).rglob('*.csv')
paths = [x for x in txt_folder]
i = 1
df = pd.DataFrame()
btc_data = preprocess_BTC()
# _, flagged_usernames = flag_users()
flagged_usernames = []
start = timer()
for path in paths:
#     if i<21:
#         i+=1
#         continue
    message = "Iterration " + str(i) + " of " + str(len(paths))
    print(message)
    text_data = preprocess_textual(path)
    
#     old
#     timetable = topic_recognition(text_data, flagged_usernames = [], display = message)
#     print(timetable)

    text = NLP_preprocess(text_data)
    text.preprocess_data()
    text.spam_filtering()
    #print(text.df.iloc[0], text.df.iloc[-1]['date'][:-2] + '00')
    print(text.df)
    text.import_tokenizer()
    df_padded = text.tokenize_and_pad(train = False)
    
    text.df["sentiment"] = (model.predict(df_padded))
    
    text.df["hour"] =  text.df.date.map(lambda x: x[:-6])
    timetable = pd.DataFrame(text.df.groupby(['hour']).sentiment.apply(np.mean).values, columns = ["Bitcoin"])
    timetable2 = pd.DataFrame(text.df.groupby(['hour']).sentiment.apply(np.median).values, columns = ["Bitcoin_median"])
    
    timetable = timetable[:-1]
    timetable2 = timetable2[:-1]
    
    btc_connected = connect_datasets(text.df, btc_data)[1:]
    
    i += 1
    
    if len(timetable.index) != len(btc_connected.index):
        continue
        
    timetable.index = btc_connected.index
    timetable2.index = btc_connected.index
    
    timetable = pd.concat([timetable['Bitcoin'], timetable2['Bitcoin_median'], btc_connected['Open']], axis = 1)
    #print(timetable)
    df = pd.concat([df, timetable], axis = 0)
    #print(df)
df.to_csv(OUTPUT_DATASET_FILEPATH)
print(timer() - start)


Iterration 1 of 65
            username                 date retweets favorites  \
7      TheActiveHive  2017-09-13 23:59:31        0         1   
284    BitcoinWorker  2017-09-13 23:45:29        0         0   
361     donizete_gil  2017-09-13 23:41:05        0         0   
417     bitcoinagile  2017-09-13 23:39:19        0         1   
432      alteregoP69  2017-09-13 23:38:46        0         0   
...              ...                  ...      ...       ...   
37647     jinggaydee  2017-09-13 00:25:17        0         0   
37829  BitcoinWorker  2017-09-13 00:15:14        0         0   
37966      syokzzzzz  2017-09-13 00:08:09        0         0   
37972  patriciawadam  2017-09-13 00:08:03        0         0   
38115         yabtcl  2017-09-13 00:00:54        0         0   

                                                    text geo mentions  \
7      earn free bitcoin faucethubio bitcoin faucet g...                
284             get free bitcoin pictwittercomqday7qmqqq          

Iterration 4 of 65
            username                 date retweets favorites  \
14        GiAgustino  2017-10-07 23:58:00        0         0   
76    Clubcryptonite  2017-10-07 23:50:49        0         0   
77    Clubcryptonite  2017-10-07 23:50:49        0         0   
78    Clubcryptonite  2017-10-07 23:50:48        0         0   
143    BitcoinWorker  2017-10-07 23:45:22        0         0   
...              ...                  ...      ...       ...   
2377  RajaNasirAyoub  2017-10-07 20:57:19        0         2   
2391  RajaNasirAyoub  2017-10-07 20:56:34        0         0   
2394         picco62  2017-10-07 20:56:18        0         0   
2395          yabtcl  2017-10-07 20:56:17        0         0   
2403  RajaNasirAyoub  2017-10-07 20:55:55        0         0   

                                                   text geo        mentions  \
14    free bitcoins faucet earn 100 free btc 5 minut...                       
76    hey david chapa thanks follow want grow bitcoi..

Iterration 7 of 65
              username                 date retweets favorites  \
65      Clubcryptonite  2017-10-13 23:55:11        1         0   
67     etownvintagebuy  2017-10-13 23:55:07        0         0   
78      RealBTCFaucets  2017-10-13 23:54:25        0         0   
118    iridzmarridzmar  2017-10-13 23:52:12        0         0   
135       city_bitcoin  2017-10-13 23:51:06        0         0   
...                ...                  ...      ...       ...   
42515       BABSWOLABI  2017-10-13 00:08:25        0         0   
42517        syokzzzzz  2017-10-13 00:08:24        0         0   
42538       BABSWOLABI  2017-10-13 00:07:16        0         0   
42706          bjdrn98  2017-10-13 00:00:31        1         0   
42738      drag_0nsden  2017-10-13 00:00:04        0         0   

                                                    text geo    mentions  \
65     hey mehodl thanks follow auto trade grow bitco...                   
67     easy free bitcoin crypto top 

Iterration 10 of 65
              username                 date retweets favorites  \
80         BoyHidetaro  2017-11-13 23:55:08        0         0   
82         Bitcoin_IRA  2017-11-13 23:55:04        0         0   
92          vanthe0712  2017-11-13 23:54:46        0         0   
130        dady_family  2017-11-13 23:53:12        0         0   
163         tangleblog  2017-11-13 23:52:02       30        74   
...                ...                  ...      ...       ...   
43631        syokzzzzz  2017-11-13 00:07:41        0         0   
43658  Supremedecentee  2017-11-13 00:05:54        0         0   
43680         topper26  2017-11-13 00:04:24        0         0   
43716           mazyja  2017-11-13 00:02:54        0         1   
43785   JereilFebrada1  2017-11-13 00:00:26        0         0   

                                                    text geo   mentions  \
80     free bitcoins faucet earn 100 free btc 5 minut...                  
82        learn bitcoins latest fork 

Iterration 13 of 65
              username                 date retweets favorites  \
47         Pro80413235  2017-11-30 23:58:17        2         2   
210     EnterTheCrypto  2017-11-30 23:53:36        1         1   
390       AfroRhapsody  2017-11-30 23:49:01        0         2   
411    bitcoins_future  2017-11-30 23:48:20        0         1   
493      SSchiffeleers  2017-11-30 23:46:13        0         0   
...                ...                  ...      ...       ...   
71726        syokzzzzz  2017-11-30 00:08:33        0         0   
71786       Chrissyshg  2017-11-30 00:07:15        0         0   
71946    DisruptBanksy  2017-11-30 00:04:12        0         0   
72010       Krister888  2017-11-30 00:03:06        3         1   
72127       Krister888  2017-11-30 00:00:57        1         1   

                                                    text geo   mentions  \
47     get 100 free coins sign bs dont miss first 100...                  
210    bitcoin rises 10000 following 

...

Iterration 53 of 65
             username                 date retweets favorites  \
24        twktter5042  2018-09-29 23:55:07        0         0   
26        twjtter5041  2018-09-29 23:55:06        0         0   
27        twwtter5040  2018-09-29 23:55:06        0         0   
29        twhtter5039  2018-09-29 23:55:05        0         0   
30        twgtter5038  2018-09-29 23:55:05        0         0   
...               ...                  ...      ...       ...   
18941    ScamAvengers  2018-09-29 00:19:16        0         0   
18994    CryptoisSexy  2018-09-29 00:15:01        0         0   
19002   newsdailytech  2018-09-29 00:14:16        0         0   
19168  JereilFebrada1  2018-09-29 00:00:17        0         0   
19195      TAGBitcoin  2018-09-29 00:00:01        0         0   

                                                    text geo mentions  \
24                                    get free bitcoin …                
26                                      get free bitc

Iterration 56 of 65
              username                 date retweets favorites  \
16          Tweets4Pay  2018-10-24 23:58:51        0         0   
32      usLibertyWatch  2018-10-24 23:57:06        0         0   
49     curatedcoinnews  2018-10-24 23:55:08        1         2   
52         twjtter5041  2018-10-24 23:55:05        0         0   
53         twwtter5040  2018-10-24 23:55:04        0         0   
...                ...                  ...      ...       ...   
23808   AileneBringino  2018-10-24 00:02:39        0         0   
23813       TAGBitcoin  2018-10-24 00:02:02        0         0   
23830       TAGBitcoin  2018-10-24 00:01:01        0         0   
23851   JereilFebrada1  2018-10-24 00:00:22        0         0   
23881       TAGBitcoin  2018-10-24 00:00:01        0         0   

                                                    text geo mentions  \
16     coinpot simple tricks earn free bitcoin altcoi...                
32     join community 35 get paid bitcoin

Iterration 59 of 65
              username                 date retweets favorites  \
8              TomVeos  2018-12-07 23:59:32        0         2   
9              TomVeos  2018-12-07 23:59:31        0         0   
152      cryptobuddha1  2018-12-07 23:47:18        0         0   
189         BitcoinTap  2018-12-07 23:44:40        0         0   
259    WIlliam62901667  2018-12-07 23:40:19        0         0   
...                ...                  ...      ...       ...   
27694       TAGBitcoin  2018-12-07 00:02:01        0         0   
27725       TAGBitcoin  2018-12-07 00:01:02        0         0   
27737        lanidoddy  2018-12-07 00:00:46        0         0   
27756   JereilFebrada1  2018-12-07 00:00:19        0         1   
27783       TAGBitcoin  2018-12-07 00:00:01        0         0   

                                                    text geo        mentions  \
8      coinpot simple tricks earn free bitcoin altcoi...                       
9      free bitcoin earning

Iterration 62 of 65
             username                 date retweets favorites  \
67         public5054  2019-01-04 23:52:51        0         0   
68         public5058  2019-01-04 23:52:51        0         0   
69         ameblo0100  2019-01-04 23:52:48        0         0   
70       suspend01cat  2019-01-04 23:52:48        0         0   
82         yumiho5120  2019-01-04 23:52:12        0         0   
...               ...                  ...      ...       ...   
21832      TAGBitcoin  2019-01-04 00:02:01        0         0   
21850      TAGBitcoin  2019-01-04 00:01:02        0         0   
21867           gt2rz  2019-01-04 00:00:35        0         0   
21874  JereilFebrada1  2019-01-04 00:00:25        0         0   
21900      TAGBitcoin  2019-01-04 00:00:01        0         0   

                                                    text geo mentions  \
67                                      get free bitcoin                
68                                      get free bitc

Iterration 65 of 65
             username                 date retweets favorites  \
31          tmatt2123  2019-01-22 22:48:20        0         1   
79       sergheipogor  2019-01-22 22:45:18        0         1   
173      swagrocky123  2019-01-22 22:38:59        0         0   
207      sergheipogor  2019-01-22 22:35:28        0         1   
308     walterpiola72  2019-01-22 22:29:24        0         0   
...               ...                  ...      ...       ...   
20937      TAGBitcoin  2019-01-22 00:02:02        0         0   
20958      TAGBitcoin  2019-01-22 00:01:01        0         0   
20977  JereilFebrada1  2019-01-22 00:00:21        0         0   
21003      TAGBitcoin  2019-01-22 00:00:02        0         0   
21004         f_toqui  2019-01-22 00:00:02        0         0   

                                                    text geo   mentions  \
31     free digibyte check abra easily invest 28 cryp...                  
79     ing bank teams r3 fiveyear blockchain deal

In [7]:
df

,Bitcoin,Bitcoin_median,Open
2017-09-13 01,0.814122,0.891900,4128.005439
2017-09-13 02,0.812894,0.850564,4066.801167
2017-09-13 03,0.853220,0.846102,3999.225167
2017-09-13 04,0.838326,0.846102,3959.409667
2017-09-13 05,0.782210,0.815073,3995.046667
...,...,...,...
2019-01-22 18,0.706585,0.819227,3584.926897
2019-01-22 19,0.889864,0.935381,3585.858545
2019-01-22 20,0.687378,0.781042,3585.295893
2019-01-22 21,0.767968,0.853416,3579.910755


In [8]:
import random
i = random.randint(0,len(text_data.index))
print(text_data.loc[i]['text'])
print(text_data.loc[i]['sentiment'])

KeyError: 135

In [ ]:
#BTC price data of the next hour standarized
Y = pd.DataFrame(df['Open']).astype(float)
#Y = dataset_standarization(Y)

#Textual data ('Bitcoin' topic popularity) of the last hour standarized
X = pd.DataFrame(df['Bitcoin']).astype(float)
#X = dataset_standarization(X)
Y.index = X.index

X = sm.add_constant(X)

model = sm.OLS(Y, X, missing = 'drop').fit()

In [ ]:
model.summary()